In [1]:
# -*- coding: UTF-8 -*-
__author__ = 'zy'
__time__ = '2020/7/25 23:08'
import requests
import math

key = '8VUjfqGwgnHwEZLxwPpnZvO1Sgeq2HFO'  # 这里填写你的百度开放平台的key
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率


def geocode(address):
    """
    利用百度geocoding服务解析地址获取位置坐标
    :param address:需要解析的地址
    :return:
    """
    geocoding = {'s': 'rsv3',
                 'key': key,
                 'city': '全国',
                 'address': address}
    res = requests.get(
        "http://restapi.amap.com/v3/geocode/geo", params=geocoding)
    if res.status_code == 200:
        json = res.json()
        status = json.get('status')
        count = json.get('count')
        if status == '1' and int(count) >= 1:
            geocodes = json.get('geocodes')[0]
            lng = float(geocodes.get('location').split(',')[0])
            lat = float(geocodes.get('location').split(',')[1])
            return [lng, lat]
        else:
            return None
    else:
        return None


def gcj02tobd09(lng, lat):
    """
    火星坐标系(GCJ-02)转百度坐标系(BD-09)
    谷歌、高德——>百度
    :param lng:火星坐标经度
    :param lat:火星坐标纬度
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]


def bd09togcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]


def wgs84togcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]


def gcj02towgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]


def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret


def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    if lng < 72.004 or lng > 137.8347:
        return True
    if lat < 0.8293 or lat > 55.8271:
        return True
    return False

def bd09towgs84(lng,lat):#114.277591,30.580842
    result2 = bd09togcj02(lng,lat)
    result4 = gcj02towgs84(result2[0], result2[1])
    return result4

import json,time
import pandas as pd
from urllib.request import urlopen, quote

def getlnglat(address):
    # http://api.map.baidu.com/geocoder/v2/?address=北京市海淀区上地十街10号&output=json&ak=您的ak&callback=showLocation //GET请求
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'
    ak = '8399wyuCP00HcW0c7rHfjMpCIT6dA4mC'  # 浏览器端密钥
    address = quote(address)  # 由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + address + '&output=' + output + '&ak=' + ak
    print(uri)
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    lat = temp['result']['location']['lat']
    lng = temp['result']['location']['lng']

    code = temp['status']
    # print(lat, lng,code,)
    return lat, lng

def get_wgs(area):
    # latlng = ('武汉·汉口火车站 ')
    geocode = getlnglat(area)
    # geocode =(39.93410210068737, 116.14756198146019)
    return bd09towgs84(geocode[1],geocode[0])
    #[116.13488415043483, 39.926555169915595]

In [3]:

area='武汉·汉口火车站 '
print(get_wgs(area))


http://api.map.baidu.com/geocoder/v2/?address=%E6%AD%A6%E6%B1%89%C2%B7%E6%B1%89%E5%8F%A3%E7%81%AB%E8%BD%A6%E7%AB%99%20&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
[116.13488415043483, 39.926555169915595]


In [6]:
bd09towgs84(114.437609,30.59992)#114.277591,30.580842  114.437609,30.59992

[114.4255257903037, 30.596501951332847]

In [2]:
with open('area.txt','r',encoding='utf-8') as f:
    areas=f.read()
print(areas.split('\n')[0:10])
areas=areas.split('\n')

['高桥二路', '高桥五路涵洞', '新沟桥铁路涵洞', '九通路铁路涵洞', '晒湖涵洞', '光谷大道金融港二路', '马场公寓', '雄楚大街省高检', '烽胜路', '江夏大道与北华街交汇处']


In [3]:
len(areas)

390

In [5]:
data={"硚口区丰顺路至园博大道涵洞":[1,2]}

datas={}

In [ ]:
import time
latlngs=[]
for i in areas:
    if i not in datas:
        try:
            latlng=get_wgs('武汉市'+i)
            datas['i']=latlng
            time.sleep(0.05)
        except:
            time.sleep(5)
            try:
                latlng=get_wgs('武汉市'+i)
                datas['i']=latlng
            except:
                latlng=[0,0]
                datas['i']=latlng
    else:
        latlng=datas['i']
    latlngs.append(latlng)

In [7]:
with open('经纬度.txt','w',encoding='utf-8') as f:
    for i in range(len(latlngs)):
        f.write(areas[i]+','+str(latlngs[i][0])+','+
                str(latlngs[i][1]))
        f.write('\n')

In [11]:
datas

{'i': [114.2995562306707, 30.595209864171355]}

In [14]:
results={}

In [12]:
set(areas)

{'21号公路',
 '21号公路石化南',
 '21号公路石化南门',
 '21号公路石化南门段',
 '万松园老三栋',
 '三店东光国际',
 '三店大道',
 '三店玉龙湾小区门口',
 '三店玉龙湾门口',
 '三店西路',
 '三店龚家湾',
 '三环线桥下南往北方向',
 '东吴大道九支沟至十支沟',
 '东新光谷大道高新四路',
 '东方马城',
 '东湖八一路阳光路口',
 '东湖路省工商局门口',
 '东湖高新光谷大道三环线',
 '东湖高新民族大道水蓝郡',
 '东西湖三店海景北区',
 '东西湖五支沟涵洞',
 '东风大道',
 '中一路涵洞',
 '中北路秦园东路',
 '中北路秦园中路',
 '中华路省粮食局门口',
 '中南路洪山广场地下通道',
 '中山大道前进一路',
 '中心城区部分地区',
 '丰盛路',
 '临江大道中华路',
 '临空港涵洞',
 '临空港铁路涵洞',
 '丽水路',
 '举水河',
 '九支沟涵洞',
 '九通路铁路涵洞',
 '书城路',
 '书城路文治街',
 '二七涵洞',
 '五琴路',
 '五角塘光谷大道雄楚大道路口',
 '交通大队周边海口二路新桥一路',
 '京汉大道前进四路',
 '光谷一路卡梅尔小镇',
 '光谷一路康威路口',
 '光谷一路金融四路',
 '光谷一路高新4路',
 '光谷一路高新四路',
 '光谷一路高新大道路口',
 '光谷一路黄龙山北路口',
 '光谷一路黄龙山隧道',
 '光谷三环线附近',
 '光谷大道三环线桥上以东',
 '光谷大道五角塘南路口',
 '光谷大道光谷一路',
 '光谷大道珞喻东路口',
 '光谷大道金融港二路',
 '光谷立交往关南园路方向',
 '光谷转盘大洋路口和民族大道路口',
 '光辉巷',
 '全力二路',
 '农大',
 '创业街关山大道路口',
 '华师南门',
 '卓刀泉北路',
 '卓刀泉南路',
 '单洞三路',
 '单洞路',
 '南湖临湖的低洼社区',
 '南湖北路天龙路口',
 '南湖周边',
 '南湖大道农科院门口',
 '南湖新城书城',
 '南湖联通渠以北社区',
 '友谊大道工业路',
 '友谊大道新生路到沙湖大桥上桥处',
 '友谊大道武车路往团结大道',
 '友谊大道湖北大学至新生路',
 '友谊路',
 '

In [16]:
for i in range(len(latlngs)):
        results[areas[i]]=[latlngs[i][0],latlngs[i][1],0]

In [17]:
for i in areas:
    results[i][2]=results[i][2]+1

In [ ]:
results

In [ ]:
with open('经纬度2016去重.txt','w',encoding='utf-8') as f:
    for i in results:
        f.write(i+','+str(results[i][0])+','+
                str(results[i][1])+','+str(results[i][2]))
        f.write('\n')

In [ ]:
bd09towgs84(114.437559,30.599719)

In [18]:
bd09towgs84(114.437559,30.599719)

{'高桥二路': [114.04614709582152, 30.622597323255757, 3],
 '高桥五路涵洞': [114.06604487275415, 30.635178369007168, 4],
 '新沟桥铁路涵洞': [114.38969033599976, 30.64536699103205, 3],
 '九通路铁路涵洞': [114.11486782043231, 30.608274782197256, 3],
 '晒湖涵洞': [114.2995562306707, 30.595209864171355, 7],
 '光谷大道金融港二路': [114.42153248861385, 30.46150124632554, 6],
 '马场公寓': [114.26346846707101, 30.615136446471706, 1],
 '雄楚大街省高检': [114.35672882257337, 30.51274548858809, 1],
 '烽胜路': [114.28760971023836, 30.489905242409343, 3],
 '江夏大道与北华街交汇处': [114.32459725306016, 30.360377741453846, 4],
 '石牌岭至省厅': [114.33581855015245, 30.52325968103402, 3],
 '光谷立交往关南园路方向': [114.41415435046783, 30.46586928357903, 3],
 '光谷一路卡梅尔小镇': [114.43530537273502, 30.42903179950928, 3],
 '江汉六桥下桥处': [114.3022269616242, 30.460307423607635, 3],
 '德才路口': [114.20099572498373, 30.573028152369503, 3],
 '隆祥东街以西': [114.2024814173584, 30.54732898840479, 2],
 '临空港铁路涵洞': [114.1339156504823, 30.65372645430043, 2],
 '文化路与谭鑫培路': [114.31844699076038, 30.3704866224131

In [20]:
with open('经纬度2016去重.txt','w',encoding='utf-8') as f:
    for i in results:
        f.write(i+','+str(results[i][0])+','+
                str(results[i][1])+','+str(results[i][2]))
        f.write('\n')

In [22]:
bd09towgs84(114.437559,30.599719)

[114.4254757736462, 30.596301509516344]